# Machine Learning project


In [5]:
pip install -r requirements.txt

  Cloning https://github.com/chlorochrule/cknn to /tmp/pip-req-build-wsl8xmq4
  Running command git clone --filter=blob:none --quiet https://github.com/chlorochrule/cknn /tmp/pip-req-build-wsl8xmq4
  Resolved https://github.com/chlorochrule/cknn to commit 7d05c5049da72a573bd486fca6647f8b0376243c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/garydoranjr/misvm.git to /tmp/pip-install-3fnb55yk/misvm_c76dfc420c444166aa160bbde23ddccf
  Running command git clone --filter=blob:none --quiet https://github.com/garydoranjr/misvm.git /tmp/pip-install-3fnb55yk/misvm_c76dfc420c444166aa160bbde23ddccf
  Resolved https://github.com/garydoranjr/misvm.git to commit b2118fe04d98c00436bdf8a0e4bbfb6082c5751c
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/tibovanheule/MINNs to /tmp/pip-req-build-j79qir78
  Running command git clone --filter=blob:none --quiet https://gi

Note: you may need to restart the kernel to use updated packages.


## MI-net

In [6]:
import numpy as np
import sys
import time
import random
from random import shuffle
import argparse

from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout

from mil_nets.dataset import load_dataset
from mil_nets.layer import Feature_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch

In [87]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    result = []
    for ibatch, batch in enumerate(test_set):
        predicted = model.predict_on_batch({'input':batch[0].astype(np.float32)})[0]

        act = list(batch[1].astype(np.float32))
        if predicted > 0.5:
            result += [1]
        else:
            result += [-1]
    return result

def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set..astype(np.float32)
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'fp':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch][0] = result[1]
    return np.mean(train_loss), np.mean(train_acc)

def MI_Net(X_train, X_test,y_train, y_test,fold):
    """Train and evaluate on MI-Net.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of MI-Net.
    """
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=50
    # load data and convert type
    train_set = []
    test_set = []
    
    batch_num = len(X_train)
    for ibag, bag in enumerate(X_train):
        batch_data = bag
        batch_label = np.array([y_train[ibag]]*len(bag))
        train_set.append((batch_data, batch_label))
        
    batch_num = len(X_test)
    for ibag, bag in enumerate(X_test):
        batch_data = bag
        batch_label = np.array([y_train[ibag]]*len(bag))
        test_set.append((batch_data, batch_label))
        
    dimension = train_set[0][0].shape[1]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout = Dropout(rate=0.5)(fc3)

    # features pooling
    fp = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp')(dropout)

    model = Model(inputs=[data_input], outputs=[fp])
    sgd = SGD(learning_rate=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss=bag_loss, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    start = time.time()
    predicted = []
    for epoch in range(max_epoch):
        print("epoch",epoch)
        train_loss, train_acc = train_eval(model, train_set)
        predicted = test_eval(model, test_set)
    time_ep = time.time() - start
    result("MI-Net", predicted,y_test, time_ep, fold, "musk1")


In [ ]:
for dataset in ['fox','mutagenesis-atoms','mutagenesis-bonds','mutagenesis-chains','eastWest','elephant','tiger','westEast','musk1']:
    example_set = parse_c45(dataset)

    # Get stats to normalize data
    raw_data = np.array(example_set.to_float())
    data_mean = np.average(raw_data, axis=0)
    data_std  = np.std(raw_data, axis=0)
    data_std[np.nonzero(data_std == 0.0)] = 1.0
    def normalizer(ex):
        ex = np.array(ex)
        normed = ((ex - data_mean) / data_std)
        if dataset == "musk1":
            normed[2:-1]
        return normed[1:-1]


    # Group examples into bags
    bagset = bag_set(example_set)

    # Convert bags to NumPy arrays
    bags = [np.array(b.to_float(normalizer)) for b in bagset]
    labels = np.array([b.label for b in bagset], dtype=float)
    # Convert 0/1 labels to -1/1 labels
    labels = 2 * labels - 1

    # perform five times 10-fold cross-validation experiments
    run = 5
    n_folds = 5

    labels = np.array(labels,dtype=int)

    bags = np.array(bags,dtype=object)

    for irun in range(run):
        fold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
        splittt = 1
        for train_index, test_index in fold.split(bags,labels):
            X_train, X_test = bags[train_index], bags[test_index]
            y_train, y_test = labels[train_index], labels[test_index]
            print('run=', irun, '  fold=', splittt)
            MI_Net(X_train, X_test,y_train, y_test,splittt)          
            splittt += 1


run= 0   fold= 0
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
run= 0   fold= 0
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
run= 0   fold= 0
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch

epoch 47
epoch 48
epoch 49
run= 3   fold= 0
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
run= 3   fold= 0
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
run= 4   fold= 0
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
ep

C:\Users\seven\Documents\github\MachineLearning\score.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(10, 10))


run= 4   fold= 0
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
run= 4   fold= 0
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
run= 4   fold= 0
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch

In [3]:
from loader import parse_c45, bag_set
from __future__ import print_function, division
from sklearn.model_selection import StratifiedKFold
from score import result
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import time


### MI-net pooling deep supervision

In [ ]:
MIimport numpy as np
import sys
import time
import random
from random import shuffle
import argparse

from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout, average

from mil_nets.dataset import load_dataset
from mil_nets.layer import Feature_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch

In [61]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net with deep supervision model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    test_loss = np.zeros((num_test_batch, 1), dtype=np.float32)
    test_acc = np.zeros((num_test_batch, 1), dtype=np.float32)
    for ibatch, batch in enumerate(test_set):
        result = model.test_on_batch({'input':batch[0].astype(np.float32)}, {'fp1':batch[1].astype(np.float32), 'fp2':batch[1].astype(np.float32), 'fp3':batch[1].astype(np.float32), 'ave':batch[1].astype(np.float32)})
        test_loss[ibatch] = result[0]
        test_acc[ibatch] = result[-1]
    return np.mean(test_loss), np.mean(test_acc)

def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net with deep supervision model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'fp1':batch[1].astype(np.float32), 'fp2':batch[1].astype(np.float32), 'fp3':batch[1].astype(np.float32), 'ave':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch] = result[-1]
    return np.mean(train_loss), np.mean(train_acc)

def MI_Net_with_DS(dataset):
    """Train and evaluate on MI-Net with deep supervision.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of MI-Net with deep supervision.
    """
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=50
    # load data and convert type
    train_bags = dataset['train']
    test_bags = dataset['test']

    # convert bag to batch
    train_set = convertToBatch(train_bags)
    test_set = convertToBatch(test_bags)
    dimension = train_set[0][0].shape[1]
    weight = [1.0, 1.0, 1.0, 0.0]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout1 = Dropout(rate=0.5)(fc1)
    dropout2 = Dropout(rate=0.5)(fc2)
    dropout3 = Dropout(rate=0.5)(fc3)

    # features pooling
    fp1 = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp1')(dropout1)
    fp2 = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp2')(dropout2)
    fp3 = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp3')(dropout3)

    # score average
    mg_ave =average([fp1,fp2,fp3], name='ave')

    model = Model(inputs=[data_input], outputs=[fp1, fp2, fp3, mg_ave])
    sgd = SGD(lr=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss={'fp1':bag_loss, 'fp2':bag_loss, 'fp3':bag_loss, 'ave':bag_loss}, loss_weights={'fp1':weight[0], 'fp2':weight[1], 'fp3':weight[2], 'ave':weight[3]}, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    t1 = time.time()
    num_batch = len(train_set)
    for epoch in range(max_epoch):
        train_loss, train_acc = train_eval(model, train_set)
        test_loss, test_acc = test_eval(model, test_set)
        print('epoch=', epoch, '  train_loss= {:.3f}'.format(train_loss), '  train_acc= {:.3f}'.format(train_acc), '  test_loss={:.3f}'.format(test_loss), '  test_acc= {:.3f}'.format(test_acc))
    t2 = time.time()
    print('run time:', (t2-t1) / 60, 'min')
    print('test_acc={:.3f}'.format(test_acc))

    return test_acc

In [62]:
# perform five times 10-fold cross=validation experiments
run = 5
n_folds = 10
acc = np.zeros((run, n_folds), dtype=float)
for irun in range(run):
    dataset = load_dataset('musk1', n_folds)
    for ifold in range(n_folds):
        print('run=', irun, '  fold=', ifold)
        acc[irun][ifold] = MI_Net_with_DS(dataset[ifold])
print('MI-Net with DS mean accuracy = ', np.mean(acc))
print('std = ', np.std(acc))

run= 0   fold= 0
[(array([[ 0.18091303, -0.8825274 , -0.4306278 , -1.2050301 , -0.16396575,
         0.22583246,  0.37227392, -1.264914  ,  0.6552512 ,  0.6779485 ,
        -1.5364    , -1.7125827 ,  0.08501184, -2.0605624 , -2.367217  ,
        -0.5470899 , -0.13566388, -0.05415298, -1.8799821 , -1.9624486 ,
         0.09283514,  0.52116567,  0.08028498,  0.6574811 ,  1.3694118 ,
         1.025903  , -0.02639023, -0.44556934,  0.38590392,  0.8780831 ,
        -0.36392334, -1.0787079 , -0.6088972 ,  1.2323903 , -1.0991892 ,
        -1.2969388 , -0.5764164 ,  1.2886021 , -0.93142754,  0.9640146 ,
        -0.9577945 , -1.2128909 ,  0.5915717 , -2.0312877 , -2.1559346 ,
        -1.1366371 , -0.6786122 ,  1.1662757 , -1.6219761 , -0.440266  ,
        -0.7766783 ,  0.4563761 ,  0.53716654, -0.93409586,  1.28761   ,
         1.0996368 ,  0.30686286, -0.77745533,  0.3906427 ,  1.2607541 ,
        -0.9363774 , -1.0160574 ,  0.5418985 ,  0.44343278, -1.3057879 ,
        -0.27241987, -0.14577623

NameError: name 'average' is not defined

## Bag-Space

In [ ]:
import pandas as pd
X = pd.read_table("./clean2.data") #pd.read_csv("sample_data/mnist_test.csv") 

In [ ]:
from cknn import cknneighbors_graph

#ckng = cknneighbors_graph(X, n_neighbors=5, delta=1.0)

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.manifold import SpectralEmbedding
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns

from cknn import cknneighbors_graph

sns.set()


def plot2d_label(X, title=None):
    digits = load_digits()
    y = digits.target
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(digits.target[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    
    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title)
    plt.show()


def main():
    data = X
    print(data)
    n_neighbors = 2

    model_normal = SpectralEmbedding(n_components=2, n_neighbors=n_neighbors)
    y_normal = model_normal.fit_transform(data)
    plot2d_label(y_normal)

    #ckng = cknneighbors_graph(data, n_neighbors=n_neighbors, delta=1.5)
    #model_cknn = SpectralEmbedding(n_components=2, affinity='precomputed')
    #y_cknn = model_cknn.fit_transform(ckng.toarray())
    #plot2d_label(y_cknn)

main()

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.manifold import SpectralEmbedding
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns

sns.set()


def plot2d_label(X, title=None):
    y = X[1]
    x_min, x_max = np.min(X[0], 0), np.max(X[0], 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title)
    plt.show()


def main():
    data = X
    n_neighbors = 10

    model_normal = SpectralEmbedding(n_components=2, n_neighbors=n_neighbors)
    y_normal = model_normal.fit_transform(data)
    #plot2d_label(y_normal)

    ckng = cknneighbors_graph(data, n_neighbors=n_neighbors, delta=1.5)
    model_cknn = SpectralEmbedding(n_components=2, affinity='precomputed')
    y_cknn = model_cknn.fit_transform(ckng.toarray())
    #plot2d_label(y_cknn)
    print(y_cknn)

main()

## instance-Space


MI-SVM and mi-SVM

In [36]:
import misvm
import time
from loader import parse_c45, bag_set
from score import result
from __future__ import print_function, division
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [2]:
# Load list of C4.5 Examples

for dataset in ['fox','mutagenesis-atoms','mutagenesis-bonds','mutagenesis-chains','eastWest','elephant','tiger','westEast', 'musk1']:
    example_set = parse_c45(dataset)


    # Get stats to normalize data
    raw_data = np.array(example_set.to_float())
    data_mean = np.average(raw_data, axis=0)
    data_std  = np.std(raw_data, axis=0)
    data_std[np.nonzero(data_std == 0.0)] = 1.0
    def normalizer(ex):
        ex = np.array(ex)
        normed = ((ex - data_mean) / data_std)
        # The ...[:, 2:-1] removes first two columns and last column,
        # which are the bag/instance ids and class label, as part of the
        # normalization process
        if dataset == "musk1":
            normed[2:-1]
        return normed[1:-1]


    # Group examples into bags
    bagset = bag_set(example_set)

    # Convert bags to NumPy arrays
    bags = [np.array(b.to_float(normalizer)) for b in bagset]
    labels = np.array([b.label for b in bagset], dtype=float)
    # Convert 0/1 labels to -1/1 labels
    labels = 2 * labels - 1

    # Construct classifiers
    classifiers = {}

    # MISVM   : the MI-SVM algorithm of Andrews, Tsochantaridis, & Hofmann (2002)
    # miSVM   : the mi-SVM algorithm of Andrews, Tsochantaridis, & Hofmann (2002)

    #  : the semi-supervised learning approach of Zhou & Xu (2007)
    #     : the MI classification algorithm of Mangasarian & Wild (2008)
    # sMIL    : sparse MIL (Bunescu & Mooney, 2007)
    # stMIL   : sparse, transductive  MIL (Bunescu & Mooney, 2007)

    classifiers['MissSVM'] = misvm.MissSVM(kernel='linear', C=1.0, max_iters=20)
    classifiers['sbMIL'] = misvm.sbMIL(kernel='linear', eta=0.1, C=1e2)
    classifiers['SIL'] = misvm.SIL(kernel='linear', C=1.0)
    classifiers['STK'] = misvm.STK(kernel='linear', C=1.0)
    classifiers['NSK'] = misvm.NSK(kernel='linear', C=1.0)
    classifiers['MICA'] = misvm.MICA(kernel='linear', C=1.0)

    # Train/Evaluate classifiers
    accuracies = {}

    bags = np.array(bags,dtype=object)
    labels = np.array(labels,dtype=int)
    fold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
    for algorithm, classifier in classifiers.items():
        nums = 1
        start = time.time()
        for train_index, test_index in fold.split(bags,labels):
            X_train, X_test = bags[train_index], bags[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            classifier.fit(X_train, y_train)
            predictions = classifier.predict(X_test)
            for i, pred in enumerate(predictions):
                if pred < 0:
                    predictions[i] = -1
                else:
                    predictions[i] = 1
            time_ep = time.time() - start
            print(algorithm, dataset)
            print(y_test)
            print(predictions)
            result(algorithm, y_test, predictions, time_ep, nums, dataset)
            accuracies[algorithm + " " + str(nums)] = {"acc":np.average(y_test == np.sign(predictions)),"kfold":nums}
            nums+=1

    for algorithm, item in accuracies.items():
        print('\n%s, fold:%s Accuracy: %.f%%' % (algorithm,str(item["kfold"]), 100 * item["acc"]))

Non-random start...
     pcost       dcost       gap    pres   dres
 0: -5.9252e+02 -2.3541e+01  1e+04  1e+02  1e-12
 1: -1.9258e+01 -2.3349e+01  2e+02  2e+00  2e-12
 2: -7.6660e+00 -1.8961e+01  3e+01  2e-01  2e-13
 3: -6.2594e+00 -1.4060e+01  1e+01  8e-02  7e-14
 4: -6.0995e+00 -8.8670e+00  3e+00  1e-02  3e-14
 5: -6.5359e+00 -7.1598e+00  7e-01  3e-03  3e-14
 6: -6.6992e+00 -6.8975e+00  2e-01  6e-04  3e-14
 7: -6.7499e+00 -6.8230e+00  8e-02  2e-04  2e-14
 8: -6.7741e+00 -6.7893e+00  2e-02  3e-05  3e-14
 9: -6.7795e+00 -6.7823e+00  3e-03  4e-06  3e-14
10: -6.7807e+00 -6.7808e+00  2e-04  1e-07  3e-14
11: -6.7807e+00 -6.7808e+00  7e-06  4e-09  3e-14
12: -6.7807e+00 -6.7807e+00  1e-07  7e-11  3e-14
Optimal solution found.

Iteration 1...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -5.9261e+02 -2.3417e+01  1e+04  1e+02  2e-12
 1: -2.1612e+01 -2.3190e+01  3e+02  3e+00  1e-12
 2: -7.8412e+00 -1.8892e+01  5e+0

Solving QP...
     pcost       dcost       gap    pres   dres
 0: -5.9183e+02 -2.3275e+01  1e+04  1e+02  1e-12
 1: -2.2162e+01 -2.3069e+01  3e+02  3e+00  1e-12
 2: -7.6879e+00 -1.9006e+01  4e+01  3e-01  2e-13
 3: -6.0554e+00 -1.2785e+01  1e+01  9e-02  5e-14
 4: -5.7847e+00 -8.1898e+00  3e+00  1e-02  3e-14
 5: -6.0438e+00 -7.1983e+00  2e+00  6e-03  3e-14
 6: -6.2405e+00 -6.6035e+00  4e-01  1e-03  3e-14
 7: -6.2906e+00 -6.5014e+00  3e-01  6e-04  3e-14
 8: -6.3408e+00 -6.4031e+00  7e-02  9e-05  3e-14
 9: -6.3566e+00 -6.3788e+00  3e-02  3e-05  3e-14
10: -6.3640e+00 -6.3676e+00  4e-03  3e-06  3e-14
11: -6.3654e+00 -6.3657e+00  4e-04  2e-07  3e-14
12: -6.3655e+00 -6.3655e+00  2e-05  1e-08  3e-14
13: -6.3655e+00 -6.3655e+00  4e-07  2e-10  3e-14
Optimal solution found.
delta obj ratio: 3.72e+04

Iteration 11...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -5.9235e+02 -2.3275e+01  1e+04  1e+02  2e-12
 1: -2.1213e

10: -6.4888e+00 -6.4894e+00  7e-04  8e-07  3e-14
11: -6.4891e+00 -6.4891e+00  2e-05  3e-08  3e-14
12: -6.4891e+00 -6.4891e+00  5e-07  5e-10  3e-14
Optimal solution found.
delta obj ratio: 1.68e+05

Iteration 20...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -5.9310e+02 -2.3162e+01  1e+04  1e+02  2e-12
 1: -2.0507e+01 -2.2950e+01  3e+02  3e+00  2e-12
 2: -7.5445e+00 -1.8637e+01  4e+01  3e-01  2e-13
 3: -5.9668e+00 -1.2078e+01  1e+01  7e-02  5e-14
 4: -5.9347e+00 -7.6672e+00  2e+00  1e-02  3e-14
 5: -6.2216e+00 -6.7196e+00  6e-01  3e-03  3e-14
 6: -6.3421e+00 -6.4893e+00  2e-01  6e-04  3e-14
 7: -6.3824e+00 -6.4245e+00  5e-02  1e-04  3e-14
 8: -6.3953e+00 -6.4058e+00  1e-02  2e-05  3e-14
 9: -6.3989e+00 -6.4010e+00  2e-03  4e-06  3e-14
10: -6.3996e+00 -6.4000e+00  5e-04  8e-07  3e-14
11: -6.3997e+00 -6.3998e+00  2e-05  2e-08  3e-14
12: -6.3998e+00 -6.3998e+00  8e-07  9e-10  3e-14
Optimal solution found.
d

MissSVM tiger
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
[ 1. -1.  1.  1. -1. -1. -1. -1.  1.  1.  1.  1. -1. -1.  1. -1. -1.  1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
  1. -1. -1. -1.]
Non-random start...
     pcost       dcost       gap    pres   dres
 0: -5.7707e+02 -2.3775e+01  1e+04  1e+02  2e-12
 1: -1.6961e+01 -2.3530e+01  2e+02  2e+00  1e-12
 2: -7.3671e+00 -1.8561e+01  3e+01  2e-01  2e-13
 3: -6.1027e+00 -1.2849e+01  1e+01  7e-02  5e-14
 4: -5.9609e+00 -8.2946e+00  3e+00  1e-02  3e-14
 5: -6.3226e+00 -6.9894e+00  7e-01  3e-03  2e-14
 6: -6.4749e+00 -6.6833e+00  2e-01  5e-04  2e-14
 7: -6.5291e+00 -6.5935e+00  7e-02  1e-04  2e-14
 8: -6.5470e+00 -6.5660e+00  2e-02  2e-05  3e-14
 9: -6.5537e+00 -6.5569e+00  3e-03  4e-06  3e-14
10: -6.5549e+00 -6.5553e+00  4e-04  2e-07  3e-14
11: -6.5550e+00 -6.5551e+00  1e-05  9e-10  3e-14
12: -6.5551e+00 -6.5551e+00  2e-07  1e-11  

Solving QP...
     pcost       dcost       gap    pres   dres
 0: -5.7694e+02 -2.3506e+01  1e+04  1e+02  2e-12
 1: -1.6286e+01 -2.3243e+01  2e+02  2e+00  1e-12
 2: -7.2467e+00 -1.8138e+01  4e+01  3e-01  2e-13
 3: -6.0347e+00 -1.2743e+01  1e+01  8e-02  7e-14
 4: -6.1412e+00 -7.5212e+00  2e+00  8e-03  3e-14
 5: -6.4095e+00 -6.9211e+00  7e-01  3e-03  3e-14
 6: -6.5164e+00 -6.7339e+00  3e-01  8e-04  3e-14
 7: -6.5628e+00 -6.6595e+00  1e-01  3e-04  3e-14
 8: -6.5812e+00 -6.6320e+00  6e-02  1e-04  3e-14
 9: -6.5947e+00 -6.6124e+00  2e-02  4e-05  3e-14
10: -6.6009e+00 -6.6038e+00  3e-03  5e-06  3e-14
11: -6.6019e+00 -6.6024e+00  6e-04  8e-07  3e-14
12: -6.6021e+00 -6.6022e+00  2e-05  2e-08  3e-14
13: -6.6021e+00 -6.6021e+00  4e-07  3e-10  3e-14
Optimal solution found.
delta obj ratio: 2.56e+05

Iteration 11...
Linearizing constraints...
Computing slacks...
Linearizing...
Solving QP...
     pcost       dcost       gap    pres   dres
 0: -5.7497e+02 -2.3513e+01  1e+04  1e+02  1e-12
 1: -1.5997e

...

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

### mi-Net

In [ ]:
import sys
import time
from random import shuffle
import numpy as np
import argparse

from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout

from mil_nets.dataset import load_dataset
from mil_nets.layer import Score_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch

In [ ]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training mi-Net model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    test_loss = np.zeros((num_test_batch, 1), dtype=np.float32)
    test_acc = np.zeros((num_test_batch, 1), dtype=np.float32)
    for ibatch, batch in enumerate(test_set):
        result = model.test_on_batch({'input':batch[0].astype(np.float32)}, {'sp':batch[1].astype(np.float32)})
        test_loss[ibatch] = result[0]
        test_acc[ibatch] = result[1]
    return np.mean(test_loss), np.mean(test_acc)

def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training mi-Net model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'sp':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch] = result[1]
    return np.mean(train_loss), np.mean(train_acc)

def mi_Net(dataset):
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=50
    """Train and evaluate on mi-Net.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of mi-Net.
    """
    # load data and convert type
    train_bags = dataset['train']
    test_bags = dataset['test']

    # convert bag to batch
    train_set = convertToBatch(train_bags)
    test_set = convertToBatch(test_bags)
    dimension = train_set[0][0].shape[1]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout = Dropout(rate=0.5)(fc3)

    # score pooling
    sp = Score_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='sp')(dropout)

    model = Model(inputs=[data_input], outputs=[sp])
    sgd = SGD(lr=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss=bag_loss, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    t1 = time.time()
    num_batch = len(train_set)
    for epoch in range(max_epoch):
        train_loss, train_acc = train_eval(model, train_set)
        test_loss, test_acc = test_eval(model, test_set)
        print('epoch=', epoch, '  train_loss= {:.3f}'.format(train_loss), '  train_acc= {:.3f}'.format(train_acc), '  test_loss={:.3f}'.format(test_loss), '  test_acc= {:.3f}'.format(test_acc))
    t2 = time.time()
    print('run time:', (t2-t1) / 60.0, 'min')
    print('test_acc={:.3f}'.format(test_acc))

    return test_acc


In [ ]:
# perform five times 10-fold cross-validation experiments
run = 5
n_folds = 10
acc = np.zeros((run, n_folds), dtype=np.float32)
for irun in range(run):
    dataset = load_dataset('musk1', n_folds)
    for ifold in range(n_folds):
        print('run=', irun, '  fold=', ifold)
        acc[irun][ifold] = mi_Net(dataset[ifold])
print('mi-net mean accuracy = ', np.mean(acc))
print('std = ', np.std(acc))